# Pràctica 1 - El procés de l'aprenentatge automàtic:
## Imports

In [1]:
import os
from sklearn.discriminant_analysis import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from skimage import io
from skimage.transform import resize
from skimage.color import rgb2gray
import numpy as np
from skimage.feature import hog
from skimage import exposure
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import joblib

## Funció de càrrega d'imatges

In [2]:
def load_images(folder_path, target_size=(128, 128)):
    images = []
    labels = []
    
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = io.imread(image_path)
            
            # Reescalar
            image = resize(image, target_size)
            # Passar a grisos
            if image.ndim == 2:
                gray_image = image
            else:
                gray_image = rgb2gray(image)
            
            images.append(gray_image)
            labels.append(label)
    
    return np.array(images), np.array(labels)

## Funció HOG

In [3]:
def hog_images(images, hog_orientations=9, hog_pixels_per_cell=(16, 16), hog_cells_per_block=(8, 8), mostrar=False):
    hog_features = []
    hog_images_rescaled = []
    for image in images:
        if(mostrar):
            # Compute HOG features
            hog_feature, hog_image = hog(image, orientations=hog_orientations,
                                        pixels_per_cell=hog_pixels_per_cell,
                                        cells_per_block=hog_cells_per_block,
                                        block_norm='L2-Hys', visualize=True)
            hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
            hog_images_rescaled.append(hog_image_rescaled)
        else:
            hog_feature = hog(image, orientations=hog_orientations,
                                        pixels_per_cell=hog_pixels_per_cell,
                                        cells_per_block=hog_cells_per_block,
                                        block_norm='L2-Hys', visualize=False)
            hog_features.append(hog_feature.flatten())
            
    if(mostrar):
        return np.array(hog_features), np.array(hog_images_rescaled)
    else:
        return np.array(hog_features)

## Càrrega de dades

In [4]:
# Carregar imatges
train_path = 'dat/train/'
test_path = 'dat/test/'
print("Carregant imatges")
X_train, y_train = load_images(train_path)
print(f"{X_train.shape} train imatges")
X_test, y_test = load_images(test_path)
print(f"{X_test.shape} test imatges")

Carregant imatges
(1500, 128, 128) train imatges
(2985, 128, 128) test imatges


## HOG

In [5]:
# Processar imatges train
print("Processant imatges train")
mostrar = False
if(mostrar):
    X_train_hog, hog_train_images = hog_images(X_train, mostrar=True)
    # Mostrar hogs
    unique_labels = np.unique(y_train)

    for label in unique_labels:
        index = np.where(y_train == label)[0][0]
        plt.imshow(hog_train_images[index], cmap='gray')
        plt.title(f'HOG Image Rescaled - Label: {label}')
        plt.show()

else:
    X_train_hog = hog_images(X_train, mostrar=False)
print(f"{X_train.shape} imatges train processades")

# Processar imatges test
print("Processant imatges test")
X_test_hog = hog_images(X_test, mostrar=False)
print(f"{X_test.shape} imatges test processades")

Processant imatges train
(1500, 128, 128) imatges train processades
Processant imatges test
(2985, 128, 128) imatges test processades


## Normalització de les dades

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_hog)
X_test_scaled = scaler.transform(X_test_hog)
print("Imatges escalades")

Imatges escalades


In [7]:
# Mostrar PCA
#n_components = 500
#pca = PCA(n_components=n_components)
#pca.fit_transform(X_train_scaled)
#explained_variance_ratio = pca.explained_variance_ratio_
#cumulative_explained_variance = np.cumsum(explained_variance_ratio)
#plt.plot(cumulative_explained_variance)
#plt.xlabel('Number of Principal Components')
#plt.ylabel('Cumulative Explained Variance')
#plt.show()

#pca = PCA()
#pca.fit_transform(X_train_scaled)
#
## Calculate the cumulative explained variance
#cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)
#
## Find the number of components that capture at least 92% of the variance
#desired_explained_variance = 0.92
#n_components_92 = np.argmax(cumulative_explained_variance >= desired_explained_variance) + 1
#
#print(f"Number of components to capture at least 92% of the variance: {n_components_92}")

# PCA per 359 components (0.92% de components que expliquen sa variança)
#pca = PCA(n_components=359)
#X_train_pca = pca.fit_transform(X_train_scaled)
#X_test_pca = pca.transform(X_test_scaled)

## Entrenament i test dels SVM

### Grid Search

### Kernel RBF

In [8]:
svm = SVC(kernel='rbf')
svm.fit(X_train_hog, y_train)
print("Model entrenat")
y_pred = svm.predict(X_test_hog)

# Evaluació model
cf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cf_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Model entrenat
Confusion matrix:
 [[ 51   4   0   3   1   0   5   0   0   2  10   0  11  28   1]
 [  7  72   0   2   1   1   2   0   0   1   7   7   8   7   0]
 [  2   2 217   7   1   3   0  39   7   9   2   0   0   0  21]
 [  3   2  11 111   1   5  16   3  12   1   5  19   4  12  10]
 [  1   1   1   6 147   5   3   0   0   2  16   4   3   2   1]
 [  1   0   2   1   2 115   3   0   3   0   4   8   1   1   0]
 [  5  11   1   9   1   1 178   1   3   0  23   6   4   5   8]
 [  0   1  40   0   0   1   0 201   1  11   4   0   0   0   1]
 [  0   0   3   5   1   0   0   0 193   0   0   0   0   0  26]
 [  1   2   7   3   2   0   0  12   0 122   1   6   0   0   4]
 [  5  12  14  16   8   4  39   5   4   5  64  16   4   7   8]
 [  1  16   1  11   7   5  20   6   1   2   8 118   6   6   0]
 [ 13   8   0   3   0   1   5   2   0   1   5   1  53  17   1]
 [ 19   4   3  11   2   1   7   1   1   0   5   4  15 116   0]
 [  2   0  29   1   0   0   2   6  10  10   3   0   0   0 211]]
Accuracy:  0.6596314

### Kernel Lineal

In [9]:
#svm = SVC(kernel='linear')
#svm.fit(X_train_scaled, y_train)
#print("Model entrenat")
#y_pred = svm.predict(X_test_scaled)
#
## Evaluate the Model
#cf_matrix = confusion_matrix(y_test, y_pred)
#print("Confusion matrix:\n", cf_matrix)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: ", accuracy)

### Kernel Polinomic

In [10]:
#svm = SVC(kernel='poly')
#svm.fit(X_train_scaled, y_train)
#print("Model entrenat")
#y_pred = svm.predict(X_test_scaled)
#
## Evaluate the Model
#cf_matrix = confusion_matrix(y_test, y_pred)
#print("Confusion matrix:\n", cf_matrix)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: ", accuracy)